# --> PIPELINE QUI scrap du calendrier des matchs à venir 
### --> construction 38 fichiers CSV 1 par journée qui liste les 10 matchs (home, away, url)
### --> assembler toutes les données dans les fichiers csv dans un seul BDD Calendrier (380 lignes)

### IMPORTER les Libraries nécessaires au fonctionnement

In [ ]:
import os
import pandas as pd
from lxml import html
import requests
from google.cloud import bigquery


### DEFINITION FONCTIONS

In [ ]:
def nom_normalizer(equipe_a_normaliser):
    teams = {
        'OLYMPIQUE LYONNAIS': ['OLYMPIQUE LYONNAIS', 'LYON', 'OL', 'O.L.'], 
        'AC AJACCIO': ['AC AJACCIO', 'AJACCIO', 'ACA', 'A.C.A', 'ATHLETIC CLUB AJACCIEN','A-C AJACCIO','A.C AJACCIO'],
        'RC STRASBOURG ALSACE' : ['STRASBOURG','RACING CLUB DE STRASBOURG ALSACE', 'RC STRASBOURG','R.C. STRASBOURG','R.C.S.','RCS', 'R.C.S.A.', 'RC STRASBOURG ALSACE', 'R.C. STRASBOURG ALSACE'], 
        'AS MONACO': ['AS MONACO', 'MONACO', 'ASSOCIATION SPORTIVE DE MONACO FOOTBALL CLUB', 'AS MONACO FC', 'A.S MONACO F.C', 'ASM'], 
        'CLERMONT FOOT 63': ['CLERMONT FOOT 63', 'CLERMONT-FERRAND', 'CF63', 'C.F.63', 'C-F 63', 'CLERMONT', 'CLERMONT FOOT'], 
        'PARIS SAINT GERMAIN': ['PARIS SAINT GERMAIN', 'PARIS-SAINT-GERMAIN FOOTBALL CLUB', 'PSG', 'P.S.G.', 'PARIS SG', 'PARIS-SG', 'PARIS SAINT-GERMAIN', 'PARIS SAINT-GERMAIN FOOTBALL CLUB', 'PARIS SAINT-GERMAIN FC'], 
        'TOULOUSE FC' : ['TOULOUSE', 'TOULOUSE FOOTBALL CLUB', 'TFC', 'T.F.C.', 'TOULOUSE FC', 'TOULOUSE F.C.'], 
        'OGC NICE': ['NICE', "OLYMPIQUE GYMNASTE CLUB NICE COTE D'AZUR", 'OGC NICE', 'O.G.C. NICE', "OGC NICE COTE D'AZUR", "O.G.C. NICE COTE D'AZUR", 'OLYMPIQUE GYMNASTE CLUB NICE'], 
        'ANGERS SCO': ['ANGERS SCO', 'ANGERS', "SCO D'ANGERS", 'SCOA', "ANGERS SPORTING CLUB DE L'OUEST", 'SCO ANGERS', 'ANGERS-SCO', 'ANGERS S.C.O.', 'S.C.O. ANGERS'], 
        'FC NANTES' : ['FC NANTES', 'NANTES', 'FOOTBALL CLUB DE NANTES', 'F.C. NANTES', 'FCN', 'F.C.N.'], 
        'RC LENS': ['RC LENS', 'LENS', 'RACING CLUB DE LENS', 'RCL', 'R.C.L.', 'R.C. LENS'], 
        'STADE BRESTOIS': ['STADE BRESTOIS', 'BREST', 'STADE BRESTOIS 29', 'SB29', 'S.B.29', 'S.B. 29'], 
        'LOSC LILLE': ['LOSC LILLE', 'LILLE', 'LOSC', 'LILLE OLYMPIQUE SPORTING CLUB', 'L.O.S.C.', 'L.O.S.C. LILLE'], 
        'AJ AUXERRE': ['AJ AUXERRE', 'AUXERRE', 'AJA', 'ASSOCIATION DE LA JEUNESSE AUXERROISE', 'A.J.AUXERRE', 'A.J AUXERRE'], 
        'MONTPELLIER HERAULT SC': ['MONTPELLIER HÃ‰RAULT SC', 'MONTPELLIER HÉRAULT SC','MONTPELLIER HÉRAULT', 'MONTPELLIER', 'MONTPELLIER-HERAULT SPORT CLUB', 'MONTPELLIER HERAULT SPORT CLUB', 'MONTPELLIER HSC', 'MHSC', 'MONTPELLIER-HERAULT SC', 'MONTPELLIER-HÉRAULT SC', 'MONTPELLIER-HERAULT S.C.', 'M.H.S.C.', 'MONTPELLIER HERAULT SC', 'MONTPELLIER HÉRAULT SC', 'MONTPELLIER HERAULT S.C.'], 
        'ESTAC TROYES': ['TROYES', 'ESPERANCE SPORTIVE TROYES AUBE CHAMPAGNE', 'ESTAC', 'E.S.T.A.C.', 'ESTAC TROYES', 'E.S.T.A.C. TROYES'], 
        'STADE RENNAIS FC': ['RENNES', 'STADE RENNAIS FOOTBALL CLUB', 'STADE RENNAIS', 'STADE RENNAIS FC', 'S.R.F.C.', 'STADE RENNAIS F.C.'], 
        'FC LORIENT': ['FC LORIENT', 'LORIENT', 'FOOTBALL CLUB LORIENT BRETAGNE SUD', 'FCL', 'F.C.L.', 'F.C. LORIENT', 'FC LORIENT BRETAGNE SUD', 'FOOTBALL CLUB LORIENTAIS'], 
        'OLYMPIQUE DE MARSEILLE': ['OLYMPIQUE DE MARSEILLE', 'MARSEILLE', 'OM', 'O.M.'],
        'STADE DE REIMS': ['STADE DE REIMS', 'REIMS', 'STADE REIMS'],
        'DIJON': ['DIJON', 'DIJON FCO', 'DFCO', "DIJON FOOTBALL COTE-D'OR", "DIJON FOOTBALL CÔTE-D'OR"],
        'SAINT ETIENNE': ['ST ETIENNE', 'ST-ETIENNE', 'ST ÉTIENNE', 'ST-ÉTIENNE', 'SAINT ETIENNE', 'SAINT-ETIENNE', 'SAINT ÉTIENNE', 'SAINT-ÉTIENNE', 'ASSE','AS SAINT ETIENNE', 'AS SAINT-ETIENNE', 'AS SAINT-ÉTIENNE', 'AS SAINT-ÉTIENNE'],
        'BORDEAUX': ['BORDEAUX', 'FOOTBALL CLUB DES GIRONDINS DE BORDEAUX', 'FC GIRONDINS DE BORDEAUX', 'GIRONDINS DE BORDEAUX', 'FCG BORDEAUX', 'FOOTBALL CLUB GIRONDINS DE BORDEAUX'],
        'METZ': ['METZ', 'FC METZ', 'FOOTBALL CLUB DE METZ', 'FOOTBALL CLUB METZ']}
    resultat = equipe_a_normaliser
    for equipe_name in teams:
        if equipe_a_normaliser in teams[equipe_name]:
            return equipe_name
    return resultat


## Paramètres à régler selon les chemins d'acces dans le GCP

In [ ]:
# nombre des jours de la saison à scrapper
nombre_jrnes = 38

# les chemins des dossiers dans le bucket
path_bucket_lake = "gs://data-lake-m2i/DB/calendrier/"
path_bucket_gold = "gs://data-lake-m2i/DB/golden/"

# TODO(developer): Set table_id to the ID of the table to create.
table_id = "involuted-disk-355708.m2i_pronos_test.calendrier-season"

## Scrap du calendrier des matchs à venir -> construction d'un fichier CSV par journée qui liste les 10 matchs (home, away, url)

In [ ]:
# journée à scrapper
for journee in range(1,nombre_jrnes+1): 
    # url de la site à scrapper
    url_a_scrapper = f"https://www.ligue1.fr/calendrier-resultats?seasonId=2022-2023&matchDay={journee}"
    # url principale de la site
    url_base = "https://www.ligue1.fr"
    # request pour avoir le corps de code de la page à scrapper
    resp = requests.get(url_a_scrapper)
    # trier la contnue du code de la page comme un arbre de html
    tree = html.fromstring(resp.content)

    # scrape toutes les id des matchs dans la page
    match_id = []
    match_id_code = tree.xpath('/html/body/main/div[3]/div[2]/div/div[2]/ul[*]/li/a/div[2]')
    for each_id in match_id_code:
        # if va chercher dans chaque balise a id son contenu 
        # et l'ajouter dans la liste de match_id
        match_id.append(each_id.attrib["id"])

    # recuperer les elements à chercher par leurs xpath
    equipe_home = []
    equipe_away = []
    match_href = []
    match_url = []
    i=0
    for each_id in match_id:
        # le resultat de tree.xpath sera en forme de liste 
        # il faut index[0] et .text pour recuperer le contenu
        equipe_home.append(nom_normalizer(tree.xpath(f'//*[@id="{each_id}_match-result"]/a/div[1]/div/span[1]')[0].text))
        equipe_away.append(nom_normalizer(tree.xpath(f'//*[@id="{each_id}_match-result"]/a/div[3]/div/span[1]')[0].text))
        # Afin d'avoir acceder à la page de detailles de match
        # on scrape le lien de chaque match et on l'ajout à la url_base
        # afin de deviner un lien complet
        # ici le resultat de tree.xpath a plusieurs attributes
        # donc on choisis l'attribue concerné par [0].attrib["href"]
        # href c'est l'attribue nous interesse
        match_href.append(tree.xpath(f'//*[@id="{each_id}_match-result"]/a'))
        match_url.append(url_base + match_href[i][0].attrib["href"])
        i+=1
    # zipper les informations des listes dans un dataframe
    # avec 3 columns correspondant à chaque liste
    # et les sauvegarder dans un fichier CSV
    df_journee_1 = pd.DataFrame(list(zip(equipe_home, equipe_away, match_url)), 
                        columns =['Equipe HOME', 'Equipe AWAY', 'URL']) 
    df_journee_1.to_csv(f'{path_bucket_lake}journee_{journee}.csv', index=False)

## Merging des csv issus du scrap calendrier -> construction d'un csv qui comprend la journée, home, away et l'url

In [ ]:
# liste vide pour accumeler dedans les dataframe de chaque journée
dataframes = []

# ajouter les données de chaque journée à la liste dataframes
for num in range(1,nombre_jrnes+1):
    df_journee = pd.read_csv(f"{path_bucket_lake}journee_{num}.csv")# storage_options={"token":"cloud"})
    df_journee.insert(0, "Journee", num)
    dataframes.append(df_journee)

# concatener la liste des dataframe et la transformer en csv dans le bucket golden
pd.concat(dataframes).to_csv(f"{path_bucket_gold}merging_calendrier.csv", index=False) #,mode="w")    


### Le csv créé (merging_calendrier) est importé dans un dataset bigQuery (en écrasant les donnnées)

In [ ]:
# Construct a BigQuery client object.
client = bigquery.Client()

job_config = bigquery.LoadJobConfig(
    skip_leading_rows=1,
    write_disposition="WRITE_TRUNCATE",
    # The source format defaults to CSV, so the line below is optional.
    source_format=bigquery.SourceFormat.CSV,
)
uri = f"{path_bucket_gold}merging_calendrier.csv"

load_job = client.load_table_from_uri(
    uri, table_id, job_config=job_config
)  # Make an API request.

load_job.result()  # Waits for the job to complete.

destination_table = client.get_table(table_id)  # Make an API request.
print("Loaded {} rows.".format(destination_table.num_rows))